In [1]:
import os
import sys
sys.path.append('../')
import hashlib
import json
from pathlib import Path
import copy

import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score, auc, roc_curve
from sklearn import metrics
import warnings
# NB: Warnings occur when computing metrics for groups with
# low sample sizes. In our case, we don't use the metrics 
# for these groups.
warnings.filterwarnings("ignore")

from compute_metrics import Model
from lib.TabularDataset import dataset_params
from lib import TabularDataset
from result_latex_utils import agg_func
# from result_latex_utils import meanPairDiff, agg_func, bold_max


# res_dir = Path('/scratch/ssd001/home/aparna/explanations-subpopulations/output_main')
res_dir = Path('../output')

In [2]:
# retain column names
retain_cols=False
list(res_dir.glob('**/*.csv'))

[WindowsPath('../output/exp0/lr_local_lime.csv')]

In [5]:
df_raw = []
for path in res_dir.glob('**/*.csv'):
    args = json.load((path.parent/'args.json').open('r'))
    # all settings for paper table 1
    #'lime','shap_blackbox_preprocessed',
    if args['explanation_type']=='local' and args['dataset'] in ['adult','compas_balanced','lsac','adult_cleaned','mimic_tab_robust','mimic_tab_fair_ds']:# and args['explanation_model'] in ['lime_test_new','lime_test','lime','shap_blackbox_preprocessed']:
        hparams = [i for i in list(args.keys()) if i not in ['seed', 'output_dir']]
        args['hparam_id'] = hashlib.md5(str([args[i] for i in hparams]).encode('utf-8')).hexdigest()
        
        
        res = pd.read_csv(path)
        
        # get performance on test set
        res = res[res.set=='test']

        if res.shape[0]>0:
            dataset = TabularDataset.Dataset(args['dataset'])
            X_train, X_train_expl, X_val_expl, X_test, y_train, y_train_expl, y_val_expl, y_test, g_train, g_train_expl, g_val_expl, g_test = dataset.get_data(
            retain_cols=retain_cols)
            
            # to check that the whole test set is present
            assert res.shape[0]==g_test.shape[0]
            res[dataset_params[args['dataset']].sensitive_attributes] = g_test.values
            assert res[res['blackbox_prob'].isna()].shape[0]==0
            assert res[res['expl_pred'].isna()].shape[0]==0
                            
            if sum(~np.isfinite(res['expl_pred'].values))>0:
                raise ValueError('For dataset {}, model {}'.format(args['dataset'],
                                                                  args['explanation_model']))
                
            # compute metrics over all items in test set
            args['accuracy_all'] = ((res['expl_pred'].values >= 0.5) == res[
                                          'blackbox_pred']).sum() / len(res)
            fpr, tpr, thresholds = roc_curve(res['blackbox_pred'].values, res['expl_pred'].values
                                             , pos_label=1)
            roc = metrics.auc(fpr, tpr)
            args['auroc_all'] = roc
            args['prevalence_all']= res['blackbox_pred'].mean()

            group_names = dataset_params[args['dataset']].sensitive_attributes
            unique_groups = res[group_names].drop_duplicates()
            for grp in group_names:
                for val in res[grp].unique():
                    unique_groups = unique_groups.append({grp: 
                                                          val, **{i: np.nan for i in group_names if i != grp}}, 
                                                         ignore_index = True)

            for group in range(unique_groups.shape[0]):
                group_i = unique_groups.iloc[group].values
                
                mask = ~pd.isnull(group_i)
                curr_group_names = np.array(group_names)[mask]
                curr_group_vals = group_i[mask]
                sel_rows1 = res[(res[np.array(group_names)[mask]].values ==
                       group_i[mask]).all(1)]
                
                # if we have 2 sensitive group columns
                if len(curr_group_names)==2:
                    sel_rows = res[(res[curr_group_names[0]]== curr_group_vals[0])&
                                   (res[curr_group_names[1]]== curr_group_vals[1])]
                elif len(curr_group_names)==1:
                    sel_rows = res[res[curr_group_names[0]]== curr_group_vals[0]]
                else:
                    print('Warning: Default setting!')
                    sel_rows = res
                
                # checking that logic of row selection works 
                assert sel_rows1.equals(sel_rows)
                
                # computing performance on given group
                group_0_val_model = Model(sel_rows['expl_pred'].values, sel_rows[
                                          'blackbox_pred'].values,
                                         sel_rows[
                                          'blackbox_prob'].values)
                all_metrics = group_0_val_model.compute()
                args_group = {
                    **copy.deepcopy(args),
                    **all_metrics
                             }
                # some metadata
                args_group['group'] = str(unique_groups.iloc[group].values)
                args_group['n'] = len(sel_rows)
                args_group['level'] = mask.sum()
                args_group['prevalence'] = sel_rows['blackbox_pred'].mean()
                args_group['pred_prevalence'] = np.mean(sel_rows['expl_pred']>=0.5)
                df_raw.append(args_group)
                # print(args_group)

df_local = pd.DataFrame(df_raw)

In [12]:
dataset_params[df_local['dataset'][0]].sensitive_attributes

['gender', 'race']

In [11]:
df_local.columns

Index(['dataset', 'blackbox_model', 'explanation_type', 'explanation_model',
       'n_features', 'model_type', 'seed', 'experiment', 'output_dir',
       'ignore_lime_weights', 'evaluate_val', 'max_epochs', 'perturb_sigma',
       'balance_groups', 'balance_labels', 'balance_group_idx',
       'train_grp_clf', 'grp_clf_attr', 'lr', 'C', 'batch_size', 'debug',
       'jtt_lambda', 'jtt_thres', 'joint_dro_alpha', 'groupdro_eta',
       'tree_depth', 'gam_max_iter', 'hparam_id', 'accuracy_all', 'auroc_all',
       'prevalence_all', 'AUROC', 'ACC', 'epsilon', 'group', 'n', 'level',
       'prevalence', 'pred_prevalence'],
      dtype='object')

In [13]:
df_local.group

0       [1. 4.]
1       [1. 2.]
2       [0. 4.]
3       [1. 1.]
4       [0. 2.]
5       [1. 3.]
6       [0. 1.]
7       [0. 0.]
8       [1. 0.]
9       [0. 3.]
10    [ 1. nan]
11    [ 0. nan]
12    [nan  4.]
13    [nan  2.]
14    [nan  1.]
15    [nan  3.]
16    [nan  0.]
Name: group, dtype: object

In [6]:
df_local

,dataset,blackbox_model,explanation_type,explanation_model,n_features,model_type,seed,experiment,output_dir,ignore_lime_weights,...,auroc_all,prevalence_all,AUROC,ACC,epsilon,group,n,level,prevalence,pred_prevalence
0,adult_cleaned,lr,local,lime,5,sklearn,1,,output\exp0,False,...,0.976128,0.188946,0.969214,0.899396,-0.037344,[1. 4.],2813,2,0.257732,0.185567
1,adult_cleaned,lr,local,lime,5,sklearn,1,,output\exp0,False,...,0.976128,0.188946,0.988466,0.959707,-0.054139,[1. 2.],273,2,0.139194,0.120879
2,adult_cleaned,lr,local,lime,5,sklearn,1,,output\exp0,False,...,0.976128,0.188946,0.991919,0.962121,-0.051018,[0. 4.],1320,2,0.077273,0.039394
3,adult_cleaned,lr,local,lime,5,sklearn,1,,output\exp0,False,...,0.976128,0.188946,0.982949,0.876289,-0.026053,[1. 1.],97,2,0.360825,0.237113
4,adult_cleaned,lr,local,lime,5,sklearn,1,,output\exp0,False,...,0.976128,0.188946,0.987124,0.975104,-0.070134,[0. 2.],241,2,0.033195,0.016598
5,adult_cleaned,lr,local,lime,5,sklearn,1,,output\exp0,False,...,0.976128,0.188946,1.000000,0.941176,-0.060338,[1. 3.],34,2,0.147059,0.088235
6,adult_cleaned,lr,local,lime,5,sklearn,1,,output\exp0,False,...,0.976128,0.188946,0.993651,0.942308,-0.073319,[0. 1.],52,2,0.134615,0.076923
7,adult_cleaned,lr,local,lime,5,sklearn,1,,output\exp0,False,...,0.976128,0.188946,1.000000,0.947368,-0.063535,[0. 0.],19,2,0.052632,0.000000
8,adult_cleaned,lr,local,lime,5,sklearn,1,,output\exp0,False,...,0.976128,0.188946,NaN,0.947368,-0.123173,[1. 0.],19,2,0.000000,0.052632
9,adult_cleaned,lr,local,lime,5,sklearn,1,,output\exp0,False,...,0.976128,0.188946,1.000000,0.882353,-0.033220,[0. 3.],17,2,0.117647,0.000000


In [14]:
# group settings
list_groups = [
    '[ 0. nan]',
    '[ 1. nan]',
    '[nan  0.]',
    '[nan  1.]',
    '[nan  2.]',
    '[nan  3.]',
    '[nan  4.]'
    
    
]

# the two sensitive groups of interest for all 4 datasets
# NB: first item in tuple is sex, second is race
# so here sex is considered the sensitive attribute for
# Adult and MIMIC and race for COMPAS and LSAC
list_groups_dict={}
list_groups_dict['adult']=[
    '[ 0. nan]',
    '[ 1. nan]']
list_groups_dict['adult_cleaned']=[
    '[ 0. nan]',
    '[ 1. nan]']
list_groups_dict['mimic_tab_robust']=[
     '[ 0. nan]',
     '[ 1. nan]']

list_groups_dict['lsac']=[
    '[nan  0.]',
    '[nan  1.]',
    '[nan  2.]',
    '[nan  3.]',
    '[nan  4.]']
list_groups_dict['lsac_cat']=[
    '[nan  0.]',
    '[nan  1.]',
    '[nan  2.]',
    '[nan  3.]',
    '[nan  4.]']

list_groups_dict['compas_balanced']=[
    '[nan  0.]',
    '[nan  1.]']



In [15]:
df_alg = []

In [16]:
# fetching hyperparams
for path in res_dir.glob('**/*.csv'):
    args = json.load((path.parent/'args.json').open('r'))
    hparams = [i for i in list(args.keys()) if i not in  ['seed', 'output_dir']]
    break

In [17]:
def get_max_gap_row(row, disp='auroc',kind='mean',return_type='str_sum',
                    gap_or_not='_gap',
                    list_groups=list_groups):
    col_names = []
    col_mean_names = []
    for group in list_groups:
        col_names.append(f'{disp}_{group}{gap_or_not}')
        col_mean_names.append((f'{disp}_{group}{gap_or_not}',kind))
    
    if return_type=='str_sum':
        max_val_ind = np.nanargmax(row[col_mean_names].values)
        val=row[col_names[max_val_ind]]
    elif return_type=='n':
        max_val_ind = np.nanargmax(row[col_mean_names].values)
        val= row[col_mean_names[max_val_ind]]
    else:
        max_val_ind = np.nanargmax(row[col_names].values)
        val= row[col_names[max_val_ind]]
    return val

In [18]:
sens_feature_dict={}
sens_feature_dict['adult_cleaned']='1'
sens_feature_dict['lsac']='2'
sens_feature_dict['mimic_tab_robust']='1'
sens_feature_dict['compas_balanced']='2'


In [19]:
# balance sensitive attribute groups by oversampling during training?
balance_group_ind = False

# balance labels by oversampling during training?
# NB: We will always do this for LSAC, see Appendix
# in paper
balance_labels_ind = False

In [20]:
df_sel=pd.DataFrame(df_raw)
df_sel=df_sel[((df_sel.dataset!='lsac')&(df_sel.balance_labels==balance_labels_ind))|
              ((df_sel.dataset=='lsac')&(df_sel.balance_labels==True))
             ]
df_sel=df_sel[df_sel.balance_groups==balance_group_ind]

In [21]:
LEVEL = 1
df=df_sel
# df=df[df.seed.isin([1,2,3,4,5])]
# df=df[(df.experiment.isin(['lime_balanced_correct_replication']))]

res_df = df.query(f'level == {LEVEL}').groupby('output_dir').apply(agg_func)

metrics = [i for i in res_df.columns if not i.startswith('worst_group')]
out1 = (res_df
     .reset_index()
     .merge(df[hparams + ['output_dir', 'seed', 'hparam_id']].drop_duplicates()))

out2 = (out1.groupby('hparam_id').agg({
   i: ('mean', 'std') for i in metrics
})
       .merge(df[hparams + ['hparam_id']].drop_duplicates().set_index('hparam_id'), left_index = True, right_index = True)
       .reset_index())

for col in metrics:
    out2[col] = out2[(col, 'mean')].apply(lambda x: '{0:.001%}'.format(x)) +' ± ' +  out2[(col, 'std')].apply(lambda x: '{0:.001%}'.format(x))


out_all=out2.copy()

In [22]:
out_all.explanation_type=='local'

0    True
Name: explanation_type, dtype: bool

In [23]:
out_all.blackbox_model.isin(['lr','nn'])

0    True
Name: blackbox_model, dtype: bool

In [24]:
out_all['dataset']

0    adult_cleaned
Name: dataset, dtype: object

In [26]:
auroc_gaps=[]
acc_gaps=[]
for sig in np.sort(out_all.perturb_sigma.unique()):
    temp1=out_all[(out_all.explanation_type=='local')&
                  (out_all.blackbox_model.isin(['lr','nn']))&
                 (out_all.perturb_sigma==sig)]
    
    # temp1=temp1[temp1.n_features.isin([100])]
    # temp1=temp1[(temp1.explanation_model=='shap_blackbox_preprocessed')|
    #            (temp1.explanation_model=='lime')|
    #            (temp1.explanation_model=='lime_test')|
    #             (temp1.explanation_model=='lime_test_new')
    #            ]
    temp1.loc[temp1.explanation_model=='shap_blackbox_preprocessed','explanation_model']='SHAP'
    temp1.loc[temp1.explanation_model=='lime','explanation_model']='LIME'
    temp1 = temp1[temp1.dataset.isin(['adult','mimic_tab_robust','lsac','adult_cleaned','compas_balanced'])]
    temp1 = temp1[temp1.explanation_model.isin(['LIME','SHAP'])]

    for perf_metric in ['auroc','accuracy','epsilon']:
        temp1[f'{perf_metric}_max_gap']=temp1.apply(lambda row: get_max_gap_row(row,perf_metric,
                                                                       list_groups=list_groups_dict[row['dataset']]), axis=1)
        temp1[f'{perf_metric}_sens_gap']=temp1.apply(lambda row: row[f'{perf_metric}_sens{sens_feature_dict[row.dataset]}_gap'],axis=1)
        
print(temp1)


                          hparam_id  (accuracy_all, mean)  \
0  f9c60bd632d6b3136ef80283c5c1c48e              0.924053   

   (accuracy_all, std)  (accuracy_min, mean)  (accuracy_min, std)  \
0                  NaN              0.899329                  NaN   

   (accuracy_minority, mean)  (accuracy_minority, std)  \
0                   0.947368                       NaN   

   (accuracy_majority, mean)  (accuracy_majority, std)  (accuracy_gap, mean)  \
0                   0.919429                       NaN              0.024724   

   ...  epsilon_[nan  1.]_gap  epsilon_[nan  2.]_gap  epsilon_[nan  3.]_gap  \
0  ...           -1.2% ± nan%            0.7% ± nan%           -0.4% ± nan%   

   epsilon_[nan  4.]_gap  auroc_max_gap  auroc_sens_gap  accuracy_max_gap  \
0           -1.3% ± nan%    0.5% ± nan%     2.0% ± nan%       2.0% ± nan%   

   accuracy_sens_gap  epsilon_max_gap  epsilon_sens_gap  
0        5.8% ± nan%     -0.0% ± nan%       1.5% ± nan%  

[1 rows x 201 columns]


In [27]:
metrics_reported_table1=['auroc_all',
                 'accuracy_max_gap',
                  'auroc_sens_gap',
                 'accuracy_sens_gap',
                 'epsilon_sens_gap'
                      ]
# temp1=temp1[temp1.explanation_model=='LIME']
pd.pivot_table(data=temp1[['dataset','blackbox_model','explanation_model']+metrics_reported_table1],
               values=metrics_reported_table1,   index = ['dataset', 'blackbox_model','explanation_model'], 
                                                    aggfunc = lambda x: x,
                                                    )

,,,accuracy_max_gap,accuracy_sens_gap,auroc_all,auroc_sens_gap,epsilon_sens_gap
dataset,blackbox_model,explanation_model,,,,,
adult_cleaned,lr,LIME,2.0% ± nan%,5.8% ± nan%,97.6% ± nan%,2.0% ± nan%,1.5% ± nan%


In [28]:
temp1=temp1[temp1.explanation_model.isin(['LIME'])]
print(temp1[['dataset','blackbox_model','explanation_model',
#              'auroc_all',
#              'auroc_max_gap',
             'accuracy_max_gap',
             'auroc_sens_gap',
             'accuracy_sens_gap',
             'epsilon_sens_gap'
             ]].sort_values(['dataset','blackbox_model','explanation_model']).to_latex())


\begin{tabular}{llllllll}
\toprule
{} &        dataset & blackbox\_model & explanation\_model & accuracy\_max\_gap & auroc\_sens\_gap & accuracy\_sens\_gap & epsilon\_sens\_gap \\
\midrule
0 &  adult\_cleaned &             lr &              LIME &      2.0\% ± nan\% &    2.0\% ± nan\% &       5.8\% ± nan\% &      1.5\% ± nan\% \\
\bottomrule
\end{tabular}

